In [413]:
print(f"InfluxDB2 client")

InfluxDB2 client


In [414]:
# InfluxDB Client docs : https://influxdb-python.readthedocs.io/en/latest/examples.html
# InfluxDB2 Client docs : https://influxdb-client.readthedocs.io/en/latest/

In [ ]:
#!pip install influxdb-client[async]

In [416]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import json
import random
import warnings
warnings.filterwarnings("ignore")

In [417]:
bucket = "SCRIPT"

In [418]:
def query_table(query_api):
    try:
        ## using Table structure
        tables = query_api.query('from(bucket:"{}") |> range(start: -5m)'.format(bucket))
        
        for table in tables:
            # print(table)
            for row in table.records:
                # print(json.dumps(row.values, indent=2))
                print(row.values)
    except Exception as e:
        print(e)

In [419]:
def query_csv(query_api):
    try:
        ## using csv library
        csv_result = query_api.query_csv('from(bucket:"{}") |> range(start: -10m)'.format(bucket))
        val_count = 0
        for row in csv_result:
            for cell in row:
                val_count += 1
    except Exception as e:
        print(e)

In [ ]:
try:
    # client = InfluxDBClient(url="https://localhost:8086", 
    #                         token="my-token", 
    #                         org="ES", 
    #                         verify_ssl=False)
    
    client = InfluxDBClient.from_config_file("influxDB2_config.ini")
    write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    sample_number = random.uniform(0,5)
    print(f"sample number : {sample_number}")
    p = Point("my_measurement").tag("location", "Prague").field("temperature", sample_number)
    
    write_api.write(bucket=bucket, record=p, write_precision='s')
except Exception as e:
    print(e)

sample number : 2.656239134571438


In [421]:
query_table(query_api)

{'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 4, 23, 20, 39, 45, 163350, tzinfo=tzutc()), '_stop': datetime.datetime(2025, 4, 23, 20, 44, 45, 163350, tzinfo=tzutc()), '_time': datetime.datetime(2025, 4, 23, 20, 44, 39, 422227, tzinfo=tzutc()), '_value': 2.058143504013508, '_field': 'temperature', '_measurement': 'my_measurement', 'location': 'Prague'}
{'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 4, 23, 20, 39, 45, 163350, tzinfo=tzutc()), '_stop': datetime.datetime(2025, 4, 23, 20, 44, 45, 163350, tzinfo=tzutc()), '_time': datetime.datetime(2025, 4, 23, 20, 44, 42, 162377, tzinfo=tzutc()), '_value': 0.9715045487922741, '_field': 'temperature', '_measurement': 'my_measurement', 'location': 'Prague'}
{'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 4, 23, 20, 39, 45, 163350, tzinfo=tzutc()), '_stop': datetime.datetime(2025, 4, 23, 20, 44, 45, 163350, tzinfo=tzutc()), '_time': datetime.datetime(2025, 4, 23, 20, 44, 43, 645022